<a href="https://colab.research.google.com/github/simulate111/Climatic_Data/blob/main/SOLCAST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests pandas

In [17]:
#@title 🛰️ Fixed: Hourly Multi-City 2024 (Safe Column Selection)
import requests
import pandas as pd
import time

# 1. API Configuration
SOLCAST_API_KEY = "Wb6HqrYpfaWokqE9kIZdVAJIc8Pu8rYS"

cities = {
    "Turku": {"lat": 60.5, "lon": 22.3},
    "Copenhagen": {"lat": 55.7, "lon": 12.6},
    "Stockholm": {"lat": 59.3, "lon": 18.1},
    "Oslo": {"lat": 59.9, "lon": 10.7}
}

months = [f"2024-{m:02d}-01" for m in range(1, 13)]
url = "https://api.solcast.com.au/data/historic/radiation_and_weather"

for city, coords in cities.items():
    all_months = []
    print(f"\n🌍 Fetching Full Data for: {city}")

    for start_date in months:
        params = {
            "latitude": coords["lat"],
            "longitude": coords["lon"],
            "api_key": SOLCAST_API_KEY,
            "start": f"{start_date}T00:00:00Z",
            "duration": "P31D",
            "period": "PT1H",
            "format": "json"
        }

        res = requests.get(url, params=params)
        if res.status_code == 200:
            all_months.append(pd.DataFrame(res.json()['estimated_actuals']))
            print(f"  ✅ {start_date} Success", end="\r")
        else:
            print(f"  ❌ {start_date} Failed: {res.status_code}")

        time.sleep(1.2)

    if all_months:
        df = pd.concat(all_months).drop_duplicates(subset=['period_end'])
        df['period_end'] = pd.to_datetime(df['period_end'])

        # --- FIXED SECTION: Safe Column Selection ---
        # We list ALL possible names Solcast might use
        potential_cols = [
            'period_end', 'ghi', 'dni', 'dhi', 'air_temp',
            'wind_speed_10m', 'wind_direction_10m', 'wind_speed_100m'
        ]

        # We only keep the columns that actually exist in the downloaded data
        existing_cols = [c for c in potential_cols if c in df.columns]

        final_df = df[existing_cols].copy()

        # Add nice formatting for your CSV
        final_df['Month'] = final_df['period_end'].dt.strftime('%B')
        final_df['Hour'] = final_df['period_end'].dt.strftime('%H:00')

        filename = f"{city}_Solar_Wind_2024.csv"
        final_df.to_csv(filename, index=False)
        print(f"\n🎉 Finished {city}! Columns found: {list(final_df.columns)}")

print("\n🚀 All downloads complete.")


🌍 Fetching Full Data for: Turku

🎉 Finished Turku! Columns found: ['period_end', 'ghi', 'dni', 'air_temp', 'Month', 'Hour']

🌍 Fetching Full Data for: Copenhagen

🎉 Finished Copenhagen! Columns found: ['period_end', 'ghi', 'dni', 'air_temp', 'Month', 'Hour']

🌍 Fetching Full Data for: Stockholm

🎉 Finished Stockholm! Columns found: ['period_end', 'ghi', 'dni', 'air_temp', 'Month', 'Hour']

🌍 Fetching Full Data for: Oslo
  ❌ 2024-03-01 Failed: 402
  ❌ 2024-04-01 Failed: 402
  ❌ 2024-05-01 Failed: 402
  ❌ 2024-06-01 Failed: 402
  ❌ 2024-07-01 Failed: 402
  ❌ 2024-08-01 Failed: 402
  ❌ 2024-09-01 Failed: 402
  ❌ 2024-10-01 Failed: 402
  ❌ 2024-11-01 Failed: 402
  ❌ 2024-12-01 Failed: 402

🎉 Finished Oslo! Columns found: ['period_end', 'ghi', 'dni', 'air_temp', 'Month', 'Hour']

🚀 All downloads complete.
